In [1]:
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import clear_output

### Almendra, Pescado Rabioso, Invisible, Spinetta Jade, Spinetta y los Socios del Desierto, Luis Alberto Spinetta
https://es.wikipedia.org/wiki/Anexo:Discografía_de_Luis_Alberto_Spinetta

 ### Spotify!

First, connect to the API with your credentials.

In [2]:
from config import SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID,
                                                      client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Let's get all his albums first.

In [27]:
artist_ids = ['3FjdJbt6Myq32uv7P4owM1'] # We start with the id for Invisible already inside... not so easy to guess the search
albums_ids = {}
artist_names = ['Almendra', 'Pescado Rabioso', 'Spinetta Jade', 'Spinetta y los Socios del Desierto', 'Luis Alberto Spinetta', 'Spinetta, García, Ferrón']
for an_artist_name in artist_names:
    an_artist_id = sp.search(q='artist:' + an_artist_name, type='artist')['artists']['items'][0]['id']
    artist_ids.append(an_artist_id)
for an_artist_id in artist_ids:
    some_albums_ids = [(x['id'], x['name']) for x in sp.artist_albums(an_artist_id, album_type='album', limit=50)['items']]
    albums_ids.update({x[1]: x[0] for x in some_albums_ids})
    

In [30]:
albums_to_disregard = ['30 Años de Almendra', 'Almendra', 'Almendra - RCA Victor 100 Años', 
                       'Antologia Almendra', 'Argentina Sorgo Films (Live At Obras / 2001)', 
                       'Elija Y Gane', 'El Album', 'Exactas (En Vivo)', 'Inolvidables RCA - 20 Grandes Exitos', 
                       'Lo Mejor de Pescado Rabioso', 'Obras Cumbres', 'San Cristóforo', 
                       'Spinetta y Las Bandas Eternas', 'Sí o Sí - Diario del Rock Argentino - Spinetta', 
                       'Tester de Violencia', 'Festival Encuentro 1977 (En Vivo)', 'Only Love Can Sustain']
for k in albums_to_disregard:
    if k in albums_ids:
        albums_ids.pop(k, None)

In [34]:
albums_ids.keys()

dict_keys(['El Jardin de los Presentes', 'Durazno Sangrando', 'Invisible', 'El 1er Álbum Más los Singles', 'Almendra 2', 'Artaud', 'Pescado Rabioso 2', 'Desatormentándonos', 'Bajo Belgrano', 'Los Niños Que Escriben En El Cielo', 'Madre En Años Luz', 'Alma de Diamante', 'Los Ojos', 'Spinetta y los Socios del Desierto', 'Un Mañana', 'La La La', 'Pan', 'Para Los Arboles', 'Silver Sorgo', 'Fuego Gris', 'Peluson Of Milk', 'Don Lucero', 'Téster de Violencia', 'Prive', 'Mondo Di Cromo', 'Kamikaze', 'Only Love Can Sustain', "A 18' Del Sol", 'Spinettalandia y Sus Amigos', 'Spinetta Los Amigo'])

__Missing "El valle interior" (1980) of Almendra__

In [70]:
spotify_raw_df = pd.DataFrame()
for an_album_title, an_album_id in albums_ids.items():
    a_dataframe = pd.DataFrame(sp.album_tracks(an_album_id)['items'])
    a_dataframe['album'] = an_album_title
    if not spotify_raw_df.shape[0]:
        spotify_raw_df = pd.DataFrame(a_dataframe)
    else:
        spotify_raw_df = pd.concat([spotify_raw_df, a_dataframe])

Deleting useless features and reseting index

In [71]:
del spotify_raw_df['available_markets']
del spotify_raw_df['explicit']
del spotify_raw_df['external_urls']
del spotify_raw_df['href']
del spotify_raw_df['is_local']
del spotify_raw_df['preview_url']
del spotify_raw_df['type']
del spotify_raw_df['id']
del spotify_raw_df['uri']

spotify_raw_df = spotify_raw_df.reset_index(drop=True)

for index, row in spotify_raw_df.iterrows():
    spotify_raw_df.at[index, 'artist'] = row['artists'][0]['name'] 
    
del spotify_raw_df['artists']
spotify_raw_df['author'] = '' # Add author for future filling
spotify_raw_df.sample(10)

,disc_number,duration_ms,name,track_number,album,artist,author
256,1,143640,Oh! Doctor,14,Fuego Gris,Luis Alberto Spinetta,
292,1,367386,La Bengala Perdida,9,Téster de Violencia,Luis Alberto Spinetta,
242,1,164066,Bahía Final,12,Silver Sorgo,Luis Alberto Spinetta,
60,1,130080,Todas las Hojas Son del Viento,1,Artaud,Pescado Rabioso,
139,1,282680,La Flor,12,Los Ojos,Spinetta Y Los Socios Del Desierto,
286,1,199880,Al Ver Verás,3,Téster de Violencia,Luis Alberto Spinetta,
204,1,251026,Un Niño Nace,18,La La La,Fito Paez,
220,1,347986,Cisne,2,Para Los Arboles,Luis Alberto Spinetta,
79,1,95533,Señorita Zapada,11,Pescado Rabioso 2,Pescado Rabioso,
116,1,133333,¿No Ves Que Ya No Somos Chiquitos?,5,Madre En Años Luz,Spinetta Jade,


Let's manually complete "El valle interior"

In [72]:
columns = ['disc_number', 'duration_ms', 'name', 'track_number', 'album', 'artist', 'author']
songs = [[1, (6*60+35)*1000, 'Las cosas para hacer', 1, "El valle interior", "Almendra", "Emilio del Guercio"],
        [1, (3*60+24)*1000, 'Amidama', 2, "El valle interior", "Almendra", "Luis Alberto Spinetta"],
        [1, (8*60+43)*1000, 'Miguelito, mi espíritu ha partido a tiempo', 3, "El valle interior", "Almendra", "Luis Alberto Spinetta"],
        [1, (2*60+3)*1000, 'Espejada', 4, "El valle interior", "Almendra", "Luis Alberto Spinetta"],
        [1, (4*60+40)*1000, 'Cielo fuerte (amor guaraní)', 5, "El valle interior", "Almendra", "Emilio del Guercio"],
        [1, (6*60+42)*1000, 'El fantasma de la buena suerte', 6, "El valle interior", "Almendra", "Luis Alberto Spinetta"],
        [1, (4*60+41)*1000, 'Buen día, día de sol', 7, "El valle interior", "Almendra", "Luis Alberto Spinetta"]]
df_to_appends = pd.DataFrame(songs, columns=columns)
raw_df = spotify_raw_df.append(df_to_appends, ignore_index=True)
raw_df.tail()

,disc_number,duration_ms,name,track_number,album,artist,author
369,1,523000,"Miguelito, mi espíritu ha partido a tiempo",3,El valle interior,Almendra,Luis Alberto Spinetta
370,1,123000,Espejada,4,El valle interior,Almendra,Luis Alberto Spinetta
371,1,280000,Cielo fuerte (amor guaraní),5,El valle interior,Almendra,Emilio del Guercio
372,1,402000,El fantasma de la buena suerte,6,El valle interior,Almendra,Luis Alberto Spinetta
373,1,281000,"Buen día, día de sol",7,El valle interior,Almendra,Luis Alberto Spinetta


### Genius!

In [73]:
def _get(path, params=None, headers=None):
    from config import GENIUS_BASE_URI, GENIUS_CLIENT_ACCESS_TOKEN
    url = '/'.join([GENIUS_BASE_URI, path])
    token = "Bearer {}".format(GENIUS_CLIENT_ACCESS_TOKEN)

    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}
    response = requests.get(url=url, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

In [88]:
# find artist ids from given data.
artist_names = ['Almendra',
 'Pescado Rabioso',
 'Spinetta Jade',
 'Spinetta y los Socios del Desierto',
 'Luis Alberto Spinetta',
 'Invisible']

for an_artist_name in artist_names:
    find_id = _get("search", {'q': an_artist_name})
    for hit in find_id["response"]["hits"]:
        if hit["result"]["primary_artist"]["name"].lower() == an_artist_name.lower():
            artist_id = hit["result"]["primary_artist"]["id"]
            break

    print("-> " + an_artist_name + "'s id is " + str(artist_id) + "\n")

-> Almendra's id is 344735

-> Pescado Rabioso's id is 357615

-> Spinetta Jade's id is 355287

-> Spinetta y los Socios del Desierto's id is 369509

-> Luis Alberto Spinetta's id is 344478

-> Invisible's id is 344478



Invisible is always problems.

In [94]:
_get("search", {'q': 'Durazno Sangrando'})["response"]["hits"][0]["result"]["primary_artist"]

{'api_path': '/artists/369283',
 'header_image_url': 'https://images.genius.com/2d07534421dc542d2a4e7a67a39ee5ac.409x409x1.jpg',
 'id': 369283,
 'image_url': 'https://images.genius.com/2d07534421dc542d2a4e7a67a39ee5ac.409x409x1.jpg',
 'is_meme_verified': False,
 'is_verified': False,
 'name': 'Invisible',
 'url': 'https://genius.com/artists/Invisible'}

Great, it's artist id is 369283

In [89]:
artist_ids = {
    'Almendra': 344735,
    'Pescado Rabioso': 357615,
    'Spinetta Jade': 355287,
    'Spinetta y los Socios del Desierto': 369509,
    'Luis Alberto Spinetta': 344478,
    'Invisible': 369283
}